In [85]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import torch, torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold

torch.manual_seed(0)
np.random.seed(0)

In [86]:
from PIL import Image, ImageDraw
import numpy as np
import json

def vector_to_numpy(drawing, side=256):
    image = vector_to_image(drawing, side)
    image_array = np.array(image)
    return image_array

def vector_to_image(drawing, side=256):
    drawing = json.loads(drawing)
    min_x, min_y, max_x, max_y = calculate_bounding_box(drawing)

    # Calculate the offset to center the drawing within the canvas
    offset_x = (side - (max_x - min_x + 1)) // 2
    offset_y = (side - (max_y - min_y + 1)) // 2

    image = Image.new('L', (side, side), color='white')  # Create a white canvas
    draw = ImageDraw.Draw(image)

    for x, y in drawing:
        xy = [(x0 - min_x + offset_x, y0 - min_y + offset_y) for x0, y0 in zip(x, y)]
        draw.line(xy, fill='black', width=1)

    return image

def calculate_bounding_box(drawing):
    all_x = [x for x, _ in drawing]
    all_y = [y for _, y in drawing]

    min_x = min(min(x) for x in all_x)
    min_y = min(min(y) for y in all_y)
    max_x = max(max(x) for x in all_x)
    max_y = max(max(y) for y in all_y)

    return min_x, min_y, max_x, max_y

In [87]:
df = pd.read_csv('doodle-detectives-aiclubiitm/train.csv')#, dtype={'drawing': np.array})
class_list = df['word'].unique()
classes = {word: index for index, word in enumerate(class_list)}
def prediction_to_words(prediction):
    return ' '.join((class_list[p] for p in prediction))
# df['drawing'] = df['drawing'].map(vector_to_numpy)

In [88]:
class BasicCNN(nn.Module):
    def __init__(self):
        nn.Module.__init__(self)
        self.convolutions = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1),
            # 1 input image. If we had an RGB image, it would be Conv2d(3, 32, 3, padding=1)
            # 32 output images, i.e, 32 kernels and 32 output images are produced
            nn.ReLU(),
            # The activation function
            nn.MaxPool2d(4),
            # Pooling with 2 x 2 blocks
            nn.Conv2d(32, 64, 3, padding=1),
            # Now we have those 32 images and we make 64 from them
            nn.ReLU(),
            nn.MaxPool2d(4)
            # Pooling again
        )
        self.fully_connected = nn.Sequential(
            nn.Flatten(),
            nn.Linear((256*256*64)//256, 600),
            # The image shape was initially 28 x 28, by pooling we've made it 7 x 7, so we divide by 16
            # We multiply by 64 because the model has learnt 64 features.
            nn.Linear(600, 128),
            nn.Linear(128, 101)
            # We have 10 output neurons (1 for each class)
        )
    def forward(self, inputs):
        # inputs = inputs.reshape([inputs.shape[1], inputs.shape[0], 256, 256])
        # print(type(inputs))
        x = self.convolutions(inputs)
        # Functions in convolution layers are run
        x = self.fully_connected(x)
        # Functions in fully connected layer are run
        return x
    def predict(self, test_loader, out='SUBMISSION.csv', out_small='sub', i_0=0): # out = None may not be implemented
        """
        Returns the predictions in a csv chosen by out, i_0 is in case you crash and have already done some stuff
        """
        self.eval()
        if not out: total_predictions=[]
        for i, (data, ids) in enumerate(iter(test_loader)):
            if i_0 > i: continue
            predictions = torch.topk(self.forward(data), 3, dim=1)[1]
            if out:
                df = pd.DataFrame((ids, predictions), columns = ['key_id', 'word'])
                df['predictions'] = df['predictions'].map(prediction_to_words)
                df.to_csv(f'{out_small}_{i}', index=False)
            else: total_predictions.append(predictions)
        if not out: return total_predictions
        else:
            total_predictions = []
            for j in range(i+1):
                total_predictions.append(pd.read_csv(f'{out_small}_{j}'))
            total_predictions = pd.concat(total_predictions)
            total_predictions.to_csv(out, index=False)

# le = preprocessing.LabelEncoder()
class MyDataset():
    def __init__(self, data, targets=None, ids=None, train=True):
        self.data = data
        if train: 
            self.targets = targets.map(lambda target: classes[target])
        if ids: self.ids = ids
    def __len__(self):
        return len(self.data)
    def __getitem__(self, i):
        img = vector_to_numpy(self.data.loc[i])
        if train: return (torch.tensor(img, dtype=torch.float32).reshape((1, *img.shape))), torch.tensor(self.targets[i], dtype=torch.int64)
        if ids: return (torch.tensor(img, dtype=torch.float32).reshape((1, *img.shape))), torch.tensor(self.ids[i], dtype=torch.int64)
        return (torch.tensor(img, dtype=torch.float32).reshape(1, *img.shape))

In [89]:
# Defining one epoch of training
def train(model, train_loader, optimizer, loss, ps=50):
    # We train the appropriate model with the input data and the appropriate optimizer
    # ps is how often we print the accuracy
    train_iter = iter(train_loader)
    model.train()
    # Puts model in train mode
    for i, (data, targets) in enumerate(train_iter):
        # i is iteration, data = 1 mini batch of images, targets = 1 mini batch target values
        # This repeats for all mini batches 
        # print(targets)
        outputs = model.forward(data) # Forward pass
        loss_val = loss(outputs, targets) # Loss computation
        # print(targets)
        optimizer.zero_grad()  # Ensures gradients stored in optimizer are reset before each backward pass
        loss_val.backward() # Backward pass
        optimizer.step() # Backward pass

        if ps and i % ps == 0:
            model.eval()
            # Puts model in evaluation mode, so we 
            with torch.no_grad():
                print(f"Loss is {loss_val}")
                predicted = outputs.max(1)[1]
                correct = (predicted == targets).sum().item()
                accuracy = correct/len(targets)
                print(f"Train accuracy is {accuracy*100:.3f}%")
def accuracy(model, test):
    # Evaluate a model given a test loader
    model.eval()
    with torch.no_grad():
        count = 0
        correct = 0
        for data, targets in iter(test):
            outputs = model.forward(data)
            predicted = outputs.max(1)[1] # Maximum output is predicted class
            count += len(targets) # Total length of datasetS
            correct += (predicted == targets).sum().item()
            # This gives a tensor of True and False values and adds no. of True values to correct each iteration
        print((predicted == targets).sum().item())
        accuracy = correct/count
        return accuracy

In [90]:
basic_cnn = BasicCNN()
cnn_optimizer = torch.optim.Adam(basic_cnn.parameters(), lr=2e-3)
loss_fn = nn.CrossEntropyLoss()

In [91]:
for train_index, val_index in StratifiedKFold(n_splits=5).split(df['drawing'], df['word']):
    df_train = df.loc[train_index].reset_index().drop('index', axis=1)
    df_val = df.loc[val_index].reset_index().drop('index', axis=1)
    break
train_dataset = MyDataset(df_train['drawing'], df_train['word'])

In [92]:
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

In [93]:
# print(next(iter(train_loader)))

[tensor([[[[255., 255., 255.,  ..., 255., 255., 255.],
          [255., 255., 255.,  ..., 255., 255., 255.],
          [255., 255., 255.,  ..., 255., 255., 255.],
          ...,
          [255., 255., 255.,  ..., 255., 255., 255.],
          [255., 255., 255.,  ..., 255., 255., 255.],
          [255., 255., 255.,  ..., 255., 255., 255.]]],


        [[[255., 255., 255.,  ..., 255., 255., 255.],
          [255., 255., 255.,  ..., 255., 255., 255.],
          [255., 255., 255.,  ..., 255., 255., 255.],
          ...,
          [255., 255., 255.,  ..., 255., 255., 255.],
          [255., 255., 255.,  ..., 255., 255., 255.],
          [255., 255., 255.,  ..., 255., 255., 255.]]],


        [[[255., 255., 255.,  ..., 255., 255., 255.],
          [255., 255., 255.,  ..., 255., 255., 255.],
          [255., 255., 255.,  ..., 255., 255., 255.],
          ...,
          [255., 255., 255.,  ..., 255., 255., 255.],
          [255., 255., 255.,  ..., 255., 255., 255.],
          [255., 255., 255.,

In [94]:
# train_loader.targets.loc[0]
df_train

,drawing,key_id,word,recognized
0,"[[[42, 15, 4, 1, 0, 6, 14, 24, 70, 72, 58, 51,...",6077534540136448,headphones,True
1,"[[[92, 96, 111, 147, 183, 211, 225, 238, 238, ...",6466812659105792,headphones,True
2,"[[[11, 8, 15, 45, 69, 81, 99, 107, 87, 52, 43,...",5836209777541120,headphones,True
3,"[[[39, 42, 56, 81, 93, 100, 115, 150, 166, 167...",6335595888508928,headphones,False
4,"[[[48, 62, 83, 129, 160, 184, 208, 216, 228], ...",5763670711730176,headphones,False
...,...,...,...,...
346082,"[[[78, 73, 73, 77, 86, 94, 108, 122, 130, 129,...",4965983666044928,mermaid,False
346083,"[[[223, 168, 119, 106, 55, 33, 20, 0, 13, 28, ...",5232698266222592,leg,False
346084,"[[[74, 65, 60, 57, 64, 74, 78, 81, 91, 128, 14...",4509987894198272,teddy-bear,False
346085,"[[[135, 106, 91, 69, 60, 31, 9, 1, 1, 14, 49, ...",4673679432613888,mermaid,True


In [95]:
train(basic_cnn, train_loader, cnn_optimizer, loss_fn)
torch.save(basic_cnn.state_dict(), 'first_model.pt')

tensor([ 10,  72,  83,  66,  79,  29,  21,  44,  37,  46,  41,   4,  76,  73,
         11,  11,  36,  66,  41,  28,  44,  85,  10,  51,  58,  67,  94,  20,
         59,  68,  79,  19,  29,  52,  30,  32,  48,  40,  52,  43,  13,   6,
         22,  13,  87,  69,  49,  52,  47,  90,  43,  74,  48,  59,  60,  91,
         63,  85,  59,  69,  91,  29,  27,  10,  78,  20,  59,  13, 100,  56,
         74,  27,  91,  99,  87,  61,   1,  55,   1,  32,   4,  63,  77,  89,
         60,  34,  18,  41,  84,  45,   5,  49,   4,   8,  22,  54,  12,  25,
         96,   4,  38,   3,  57,  19,  38,  43,  21,   2,  25,  37,  32,  70,
         93,  28,   3,  80,  23,  69,  43,  12,  79,  75,  12,  56,  50,  74,
         43,  39])
Loss is 19.361553192138672
Train accuracy is 0.000%
tensor([90,  7, 92, 12, 24, 95, 37, 59, 31, 85,  3, 13, 26, 60, 64, 19, 89, 85,
        33, 73, 63, 87, 44, 20, 98, 70, 30, 83, 24,  5, 20, 70, 99, 85, 51, 99,
        69, 20, 82, 19, 83, 24, 75, 88, 65, 14, 85, 56, 34, 27,  1,

KeyboardInterrupt: 

In [ ]:
val_set = MyDataset(df_val['drawing'], df_val['word'])
val_loader = DataLoader(val_set, batch_size=128)
print(accuracy(basic_cnn, val_loader))

4
0.013020833333333334


In [ ]:
dft = pd.read_csv('doodle-detectives-aiclubiitm/test.csv')#, dtype={'drawing': np.array})
# dft['drawing'] = dft['drawing'].map(vector_to_numpy)


In [ ]:
test_dataset = MyDataset(dft['drawing'], train=False, ids=dft['key_id'])
test_loader = DataLoader(test_dataset, batch_size=128, drop_last=False)

RuntimeError: [enforce fail at ..\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 48611721216 bytes.

In [ ]:
basic_cnn.load_state_dict(torch.load('first_model.pt'))
basic_cnn.predict(test_loader)
# basic_cnn.eval()
# with torch.no_grad():
#     count = 0
#     correct = 0
#     for data, targets in iter(dataloader):
#         outputs = model.forward(data)
#         predicted = outputs.max(1)[1] # Maximum output is predicted class
#         count += len(targets) # Total length of datasetS
#         correct += (predicted == targets).sum().item()
#         # This gives a tensor of True and False values and adds no. of True values to correct each iteration
#     print((predicted == targets).sum().item())
#     accuracy = correct/count
#     return accuracy
